In [ ]:
!pip install paddlelite
!pip install paddlex==1.3.11
!pip list |grep paddle

In [4]:
defect_name2label = {
    '破洞': 1, '水渍': 2, '油渍': 2, '污渍': 2, '三丝': 3, '结头': 4, '花板跳': 5, '百脚': 6, '毛粒': 7,
    '粗经': 8, '松经': 9, '断经': 10, '吊经': 11, '粗维': 12, '纬缩': 13, '浆斑': 14, '整经结': 15, '星跳': 16, '跳花': 16,
    '断氨纶': 17, '稀密档': 18, '浪纹档': 18, '色差档': 18, '磨痕': 19, '轧痕': 19, '修痕': 19, '烧毛痕': 19, '死皱': 20, '云织': 20,
    '双纬': 20, '双经': 20, '跳纱': 20, '筘路': 20, '纬纱不良': 20,
}

In [5]:
import os
import pandas as pd
import numpy as np
import shutil
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx

In [6]:
from paddlex.det import transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(),
    transforms.ResizeByShort(short_size=608, max_size=608),
    transforms.Padding(coarsest_stride=32),
    transforms.Resize(target_size=512, interp='LINEAR')
])

eval_transforms = transforms.Compose([
    transforms.Normalize(),
    transforms.ResizeByShort(short_size=608, max_size=608),
    transforms.Padding(coarsest_stride=32),
    transforms.Resize(target_size=512, interp='LINEAR')# 转换照片大小 YOLO要求照片大小为32的倍数
])

In [ ]:
train_dataset = pdx.datasets.CocoDetection(
    data_dir='./train_dataset',
    ann_file='./instances_train.json',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.CocoDetection(
    data_dir='./val_dataset',
    ann_file='instances_val.json',
    transforms=eval_transforms)

In [8]:
num_classes = len(train_dataset.labels)
#model = pdx.det.FasterRCNN(backbone='ResNet50_vd',with_fpn=False,num_classes=num_classes)
model = pdx.det.PPYOLO(num_classes=num_classes, backbone='ResNet50_vd_ssld', with_dcn_v2=False,)

In [ ]:
model.train(num_epochs=240, 
            train_dataset=train_dataset, 
            train_batch_size=8, eval_dataset=eval_dataset, 
            save_interval_epochs=10, log_interval_steps=200,
             save_dir='output/PPYOLO', optimizer=None, learning_rate=1.0/8000, use_ema=True, ema_decay=0.9998)

In [ ]:
model = pdx.load_model('output/PPYOLO/best_model')#加载模型
model.evaluate(eval_dataset, batch_size=1, epoch_id=None, metric=None, return_details=False)
eval_details_file = 'output/PPYOLO/best_model/eval_details.json'
pdx.det.draw_pr_curve(eval_details_file, save_dir='./fabric_flaw_det')

In [ ]:
import json 
import os
import paddlex as pdx
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
predictor = pdx.deploy.Predictor('inference_model/PPYOLO')
testdata_path = "test_dataset"
namelist = os.listdir(testdata_path)

In [ ]:
result = [] 
for i in namelist:
    if i == ".ipynb_checkpoints" or i == "Untitled Folder":
        continue
    cur = predictor.predict(image=os.path.join(testdata_path,i))
    cur = sorted(cur,key = lambda e:e['score'],reverse = True)
    if len(cur) == 0 : 
        continue
    category_int = int(cur[0]['category'])
    x1,y1,x2,y2 = cur[0]['bbox'][0], cur[0]['bbox'][1], cur[0]['bbox'][0] + cur[0]['bbox'][2], cur[0]['bbox'][1] + cur[0]['bbox'][3]
    result.append({'name':i,'category':category_int,'bbox':[x1,y1,x2,y2],'score': cur[0]['score']}) 

with open('result.json', 'w') as fp: 
    json.dump(result, fp, indent=4, separators=(',', ': '))